In [30]:
!pip install --use-pep517 utils


In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import time

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [58]:
train_dir = "data/drug_users_train"   # contains 'drug_user' and 'not_user'
test_dir = "data/drug_users_test"     # contains 'drug_user' and 'not_user'

In [59]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])


In [60]:
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Classes:", train_dataset.classes) 

Classes: ['drug_user', 'not_user']


In [61]:
model = models.efficientnet_b0(weights="IMAGENET1K_V1")

# Replace classifier for binary classification
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)

model = model.to(device)

In [62]:
class_counts = torch.tensor([240, 160], dtype=torch.float)  # [drug_user, not_user] in training
class_weights = 1.0 / class_counts  # inverse frequency
class_weights = class_weights / class_weights.sum()  # normalize to sum=1
class_weights = class_weights.to(device)

print("Class Weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Class Weights: tensor([0.4000, 0.6000])


In [63]:
def train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=15):
    best_acc = 0.0
    print("\n========== Start Training ==========\n")

    for epoch in range(num_epochs):
        model.train()
        running_loss, running_corrects = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        # Validation
        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(test_dataset)

        print(f"Epoch {epoch+1}/{num_epochs} "
              f"Train Loss: {epoch_loss:.4f} "
              f"Train Acc: {epoch_acc:.4f} "
              f"Val Acc: {val_acc:.4f}")

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

    print(f"\nTraining finished. Best Validation Accuracy: {best_acc:.4f}")
    return model


In [ ]:
trained_model = train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=20)



========== Start Training ==========



In [ ]:
trained_model.load_state_dict(torch.load("best_model.pth"))
trained_model.eval()